# Qwen3-TTS 파인튜닝 - Debi Voice (0.6B)

**사전 준비:**
- Google Drive에 `debi_tts_data` 폴더 업로드
- 폴더 구조: `debi_tts_data/debi_finetune.jsonl` + `debi_tts_data/audio/*.wav`
- 런타임 유형: A100 GPU

**0.6B 변경사항 (v6 기반):**
- flash_attention_2 사용 (A100 최적화)
- 초저학습률 (1e-6) - 모델 파괴 방지
- batch_size 8 - 학습 안정성 향상
- num_epochs 5 - 과적합 방지

## 셀 1: 환경 설정

In [ ]:
!apt-get install -y sox
!pip install -q soundfile librosa tqdm huggingface_hub
!pip install flash-attn --no-build-isolation
!git clone https://github.com/QwenLM/Qwen3-TTS.git /content/Qwen3-TTS-repo
%cd /content/Qwen3-TTS-repo
!pip install -e .
print("환경 설정 완료! (flash_attn 포함)")

## 셀 2: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 이전 모델 삭제 (공간 확보)
!rm -rf /content/drive/MyDrive/debi_model_0.6b
print("드라이브 마운트 완료!")

## 셀 3: 학습 코드 패치 (tensorboard만 제거)

In [ ]:
# tensorboard 제거 + 0.6B text_projection 패치
sft_path = "/content/Qwen3-TTS-repo/finetuning/sft_12hz.py"

with open(sft_path, 'r') as f:
    code = f.read()

# tensorboard -> None (Colab에서 에러 방지)
code = code.replace('log_with="tensorboard"', 'log_with=None')

# 0.6B 호환: text_projection 추가 (Issue #39)
# 1.7B는 text_embedding(2048) == codec_embedding(2048)이라 문제없지만
# 0.6B는 text_embedding(2048) != codec_embedding(1024)이므로 projection 필요
code = code.replace(
    'input_text_embedding = model.talker.model.text_embedding(input_text_ids) * text_embedding_mask',
    'input_text_embedding = model.talker.text_projection(model.talker.model.text_embedding(input_text_ids)) * text_embedding_mask'
)

with open(sft_path, 'w') as f:
    f.write(code)

print("sft_12hz.py 패치 완료!")
print("변경사항: log_with=None, text_projection 추가 (0.6B 호환)")

## 셀 4: 모델 다운로드 (config 수정 없음)

In [ ]:
from huggingface_hub import snapshot_download

# 0.6B Base 모델 다운로드 (추론 속도 개선용)
model_path = snapshot_download(
    "Qwen/Qwen3-TTS-12Hz-0.6B-Base", 
    local_dir="/content/qwen3_tts_model"
)

print(f"모델 준비 완료: {model_path}")
print("flash_attention_2 설정 유지")

## 셀 5: 오디오 24kHz 변환

In [ ]:
import os, librosa, soundfile as sf
from tqdm import tqdm

AUDIO_DIR = "/content/drive/MyDrive/debi_tts_data/audio"
OUTPUT_DIR = "/content/audio_24k"
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]
skipped = []
for f in tqdm(files):
    try:
        audio, _ = librosa.load(os.path.join(AUDIO_DIR, f), sr=24000)
        sf.write(os.path.join(OUTPUT_DIR, f), audio, 24000)
    except:
        skipped.append(f)

print(f"완료: {len(files)-len(skipped)}개, 스킵: {len(skipped)}개")

## 셀 6: JSONL 경로 업데이트

In [ ]:
import json, os

with open("/content/drive/MyDrive/debi_tts_data/debi_finetune.jsonl", 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

valid_files = set(os.listdir("/content/audio_24k"))

# 모든 샘플에 동일한 ref_audio 사용 (공식 문서 권장)
REF_AUDIO = "/content/audio_24k/Debi_airSupply_2_01.wav"

filtered = []
for item in data:
    filename = item['audio'].split('/')[-1]
    if filename in valid_files:
        item['audio'] = f"/content/audio_24k/{filename}"
        item['ref_audio'] = REF_AUDIO
        filtered.append(item)

with open("/content/debi_24k.jsonl", 'w', encoding='utf-8') as f:
    for item in filtered:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"JSONL 생성: {len(filtered)}개")
print(f"ref_audio (모든 샘플 동일): {REF_AUDIO}")

## 셀 7: 데이터 전처리 (토큰화)

In [ ]:
!python /content/Qwen3-TTS-repo/finetuning/prepare_data.py \
    --device cuda:0 \
    --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz \
    --input_jsonl /content/debi_24k.jsonl \
    --output_jsonl /content/debi_tokenized.jsonl

## 셀 8: 파인튜닝 실행 (안정화 설정)

In [ ]:
# v6: 모델 파괴 방지를 위한 안정화 설정
# - batch_size 8: A100 성능 활용, 학습 안정성 확보
# - lr 1e-6: 초저학습률로 기존 지식 보존
# - num_epochs 5: 과적합 방지

!python /content/Qwen3-TTS-repo/finetuning/sft_12hz.py \
    --init_model_path /content/qwen3_tts_model \
    --train_jsonl /content/debi_tokenized.jsonl \
    --output_model_path /content/drive/MyDrive/debi_model_0.6b \
    --batch_size 8 \
    --lr 1e-6 \
    --num_epochs 5 \
    --speaker_name debi

## 셀 9: 파인튜닝 모델 테스트

In [ ]:
import torch
import soundfile as sf
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

device = "cuda:0"
tts = Qwen3TTSModel.from_pretrained(
    "/content/drive/MyDrive/debi_model_0.6b/checkpoint-epoch-4",
    device_map=device,
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

print("Speakers:", tts.get_supported_speakers())

wavs, sr = tts.generate_custom_voice(
    text="안녕하세요, 데비입니다! 오늘도 좋은 하루 보내세요.",
    speaker="debi",
)

display(Audio(wavs[0], rate=sr))
sf.write("/content/debi_test.wav", wavs[0], sr)
print("파인튜닝 모델 테스트 완료!")

## 셀 10: Base 모델 Voice Clone 비교

In [ ]:
import torch
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

base_model = Qwen3TTSModel.from_pretrained(
    "/content/qwen3_tts_model",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

ref_audio = "/content/audio_24k/Debi_airSupply_2_01.wav"
ref_text = "줄 거면 좀 쉽게 열리게 만들면 덧나?"

prompt = base_model.create_voice_clone_prompt(ref_audio=ref_audio, ref_text=ref_text)
wavs, sr = base_model.generate_voice_clone(
    text="안녕하세요, 데비입니다! 오늘도 좋은 하루 보내세요.",
    voice_clone_prompt=prompt,
)

display(Audio(wavs[0], rate=sr))
print("Base model voice clone 테스트 완료!")